In [1]:
include("./bayes.jl")
include("./helpermodules.jl")
using Gtk, Gtk.ShortNames ,Word2Vec, LIBSVM, JLD, SVR, .Bayes, .Sentiment140DataFrame, .TweetFormat
style_file = joinpath(dirname(Base.source_path()), "style.css")
provider = CssProviderLeaf(filename = style_file)

GtkCssProviderLeaf()

In [2]:
word_model = wordvectors("C:\\Users\\joshc\\text8-vec.txt")
function createFeatureVector(sentence)
    wordCount = 0
    vector = zeros(size(word_model)[1])
    sentence_split = split(sentence)
    for word in sentence_split
        try
            vector =  vector + get_vector(word_model, word)
            wordCount = wordCount + 1
        catch
            nothing
        end
    end 
    if wordCount > 0
        vector = vector / wordCount
    end
    return vector
end

createFeatureVector (generic function with 1 method)

### Load ML Model

In [3]:
model = loadnb("naivebayes")

NaiveBayes(8604079, 9374817, Dict{AbstractString,Int64}("beyb" => 1,"rearrangement" => 2,"frogger" => 2,"wtccs" => 2,"aunie" => 1,"twighlight" => 5,"yallah" => 1,"grp" => 6,"montery" => 1,"swimms" => 1…), Dict{AbstractString,Int64}("beyb" => 3,"canclled" => 2,"frogger" => 3,"collegealready" => 1,"twighlight" => 3,"tunnell" => 1,"mysqls" => 1,"grp" => 13,"yallah" => 1,"snowborder" => 1…))

### The Following is the main menu

In [4]:
mainWin = GtkWindow("Home", 400, 295,visible = false)
vboxMain = GtkBox(:v)
hboxMain1 = GtkBox(:h)
hboxMain2 = GtkBox(:h)
sentimentID = GtkButton("\n\n    Sentiment Identifier      \n\n")
set_gtk_property!(sentimentID, :name, "b1")
screen = Gtk.GAccessor.style_context(sentimentID)
push!(screen, StyleProvider(provider), 600)
similarPhrase = GtkButton("\n\n     Find a similar phrase    \n\n")
posVsNeg = GtkButton("\n\nPlot positive & negative words\n\n")
modelComparison = GtkButton("\n\nView model accuracy \n\n")
mainMenuLabel = GtkLabel("Main Menu")
push!(hboxMain1,sentimentID)
push!(hboxMain1,similarPhrase)
push!(hboxMain2, posVsNeg)
push!(hboxMain2, modelComparison)
push!(vboxMain, mainMenuLabel)
push!(vboxMain, hboxMain1)
push!(vboxMain, hboxMain2)
push!(mainWin, vboxMain)

GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=FALSE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Home", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=400, default-height=295, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint=FALSE

### The Following is for the sentiment analysis feature

In [5]:
sentWin = GtkWindow("Sentiment Indentifier", 450,200,visible = false)
prompt = GtkLabel("")
GAccessor.markup(prompt,"""<b>Please enter in a sentence or phrase that you wish to get the sentiment for below.</b>\n""")
#User entry
ent = GtkEntry()
set_gtk_property!(ent,:text, "Enter text here... ")

# set_gtk_property!(ent,:default-width, 400)
enterButton = GtkButton("Enter")
hbox = GtkBox(:h)
push!(hbox, ent)
push!(hbox, enterButton)

label = GtkLabel("Your text contains a ... sentiment.")
vbox = GtkBox(:v)
homeButton = GtkButton("Home")
push!(vbox,prompt)
push!(vbox, hbox)
push!(vbox, label)
push!(vbox, homeButton)
push!(sentWin, vbox)
function on_enter_click(w)
    str = get_gtk_property(ent,:text,String)
    if predict_positive(str, model) == true
        GAccessor.text(label,"Your text contains a positive sentiment.")
    else
        GAccessor.text(label,"Your text contains a negative sentiment.")
    end
end
signal_connect(on_enter_click, enterButton,"clicked")

0x000000a0

### The Following is for similar phrase feature

In [6]:
simPhraseWin = GtkWindow("Similar Phrase", 400, 400,visible = false)

GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=FALSE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Similar Phrase", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=400, default-height=400, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-

### The Following is for ploting positive and negative words 

In [7]:
posNegWin = GtkWindow("Positve and Negative Word Plot", 400,400,visible = false)

GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=FALSE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Positve and Negative Word Plot", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=400, default-height=400, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMA

### The Following is for displaying model accuracy and comparions 

In [8]:
modelDisplayWin = GtkWindow("Display Model Accuracy", 400, 400,visible = false)

GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=FALSE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Display Model Accuracy", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=400, default-height=400, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-

### Functions for changing windows

In [9]:
function on_sentID_click(w)
    visible(mainWin, false)
    showall(sentWin)
end
function on_simPhrase_click(w)
    visible(mainWin, false)
    showall(simPhraseWin)
end
function on_PosNeg_click(w)
    visible(mainWin, false)
    showall(posNegWin)
end
function on_modelDisp_click(w)
    visible(mainWin, false)
    showall(modelDisplayWin)
end
function on_home_click(w)
    visible(sentWin, false)
    showall(mainWin)
end   
signal_connect(on_home_click, homeButton, "clicked")
signal_connect(on_sentID_click, sentimentID,"clicked")
signal_connect(on_simPhrase_click, similarPhrase,"clicked")
signal_connect(on_PosNeg_click, posVsNeg,"clicked")
signal_connect(on_modelDisp_click, modelComparison,"clicked")

0x000000fc

In [10]:
showall(mainWin)  


GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Home", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=400, default-height=295, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint=FALSE,